In [1]:
import pandas as pd

phase_thr=pd.read_excel('C:\\Users\\ajou\\Desktop\\Data\\dongtan\\signal.xlsx', sheet_name='phase_thr')
phase_left=pd.read_excel('C:\\Users\\ajou\\Desktop\\Data\\dongtan\\signal.xlsx', sheet_name='phase_left')
phase_weekdays=pd.read_excel('C:\\Users\\ajou\\Desktop\\Data\\dongtan\\signal.xlsx', sheet_name='phase_weekdays')
phase_weekend=pd.read_excel('C:\\Users\\ajou\\Desktop\\Data\\dongtan\\signal.xlsx', sheet_name='phase_weekend')
phase_time_thr=pd.read_excel('C:\\Users\\ajou\\Desktop\\Data\\dongtan\\signal.xlsx', sheet_name='phase time_thr')
phase_time_left=pd.read_excel('C:\\Users\\ajou\\Desktop\\Data\\dongtan\\signal.xlsx', sheet_name='phase time_left')

In [2]:
columns=[40221,  40222,  40223,  40224,  40231,  40232,  40233,  40234,
        40241,  40242,  40243,  40244,  40311,  40312,  40313,  40314,  40321,
        40322,  40323,  40324,  40331,  40332,  40333,  40334,  40341,  40342,
        40343,  40351,  40352,  40353,  40354,  40371,  40372,  40373,
        40374,  40381,  40382,  40383,  40384,  40391,  40392,  40393,  40394]


#40344 공사중인 링크 제거

In [3]:
from datetime import datetime, timedelta

# 시작 날짜와 시간 설정
start_date = datetime(2023, 1, 1)
current_date = start_date

# 초단위로 증가하는 컬럼을 저장할 리스트 생성
time_column = []

# 데이터프레임의 행 수
num_rows = 2678400
#num_rows = 1000

# 초단위로 증가하는 컬럼 생성
for _ in range(num_rows):
    time_column.append(current_date)
    current_date += timedelta(seconds=1)

# 데이터프레임 생성
data = {'Timestamp': time_column}
df_time = pd.DataFrame(data)

In [4]:
# 데이터프레임 복사 (깊은 복사)
df = df_time.copy()
df_offset = df_time.copy()

# 복사된 데이터프레임에 작업 수행
df[[columns]] = ''
df_offset[[columns]] = ''

# df와 df_offset이 서로 다른 데이터프레임을 가리키게 됩니다.       
# 컬럼추가 & offset 적을 데이터프레임 생성

In [5]:
df

,Timestamp,40221,40222,40223,40224,40231,40232,40233,40234,40241,...,40373,40374,40381,40382,40383,40384,40391,40392,40393,40394
0,2023-01-01 00:00:00,,,,,,,,,,...,,,,,,,,,,
1,2023-01-01 00:00:01,,,,,,,,,,...,,,,,,,,,,
2,2023-01-01 00:00:02,,,,,,,,,,...,,,,,,,,,,
3,2023-01-01 00:00:03,,,,,,,,,,...,,,,,,,,,,
4,2023-01-01 00:00:04,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2678395,2023-01-31 23:59:55,,,,,,,,,,...,,,,,,,,,,
2678396,2023-01-31 23:59:56,,,,,,,,,,...,,,,,,,,,,
2678397,2023-01-31 23:59:57,,,,,,,,,,...,,,,,,,,,,
2678398,2023-01-31 23:59:58,,,,,,,,,,...,,,,,,,,,,


In [6]:
pd.set_option('mode.chained_assignment', None)  # 경고 비활성화

In [7]:
def filter_rows(plan,phase,time,link):
    search_value=link
    intersection_row= phase[phase.applymap(lambda cell: str(search_value) in str(cell)).any(axis=1)]
    
    column_value=[]
    for column in intersection_row.columns:
        if (intersection_row[column].apply(lambda cell: str(search_value) in str(cell))).any():
              
            column_value.append(column)
    
    intersection=intersection_row.iloc[0,0]
    filtered_rows = plan[(plan['time'] == time) & (plan['intersection'] == intersection)]
    
    return filtered_rows,column_value

In [8]:
#아래 함수에 plan 안들어가니까 다 끝나면 지우자
#일단 직진만 계산 중임

In [9]:
def calculate_signal(df, plan, link, df_offset):
    signal_data = []

    for i in range(df.shape[0]): #초마다 반복
        date_value = df.iloc[i, 0]
        weekday = date_value.weekday()

        if weekday >= 5:
            selected_plan = phase_weekend
        else:
            selected_plan = phase_weekdays

        hour = date_value.hour

        if hour >= 5:
            selected_time = 5
        elif hour >= 10:
            selected_time = 10
        elif hour >= 16:
            selected_time = 16
        else:
            selected_time = 22

        filtered_plan, phase_time = filter_rows(selected_plan, phase_thr, selected_time, link)
        offset, t1, t2, t3, t4, t5, t6 = filtered_plan.iloc[0, 2:9]

        j = i % (t1 + t2 + t3 + t4 + t5 + t6)

        phase_cycle = [t1, t2, t3, t4, t5, t6]
        current_phase_index = 0

        for cycle in phase_cycle:
            if j < cycle:
                phase_name = f't{current_phase_index + 1}'
                break
            j -= cycle
            current_phase_index += 1

        signal_value = 1 if phase_name in phase_time else 0
        signal_data.append(signal_value)
        
         # offset 값을 해당 시간에 저장
        df_offset.at[i, link] = offset

    return signal_data


In [10]:
# 신호 계산

for link in columns:
    df[link] = calculate_signal(df, phase_weekdays, link, df_offset)

In [ ]:
df_grouped=df.groupby(pd.Grouper(key='Timestamp', freq='5T')).sum()


In [ ]:
df_grouped_offset=df_offset.groupby(pd.Grouper(key='Timestamp', freq='5T')).mean()

In [ ]:
df_grouped

In [ ]:
df_grouped_offset

In [ ]:
df_grouped.to_excel('C:\\Users\\ajou\\Desktop\\Data\\dongtan\\signal_green.xlsx')
df_grouped_offset.to_excel('C:\\Users\\ajou\\Desktop\\Data\\dongtan\\signal_offset.xlsx')